[View in Colaboratory](https://colab.research.google.com/github/dkanunnikov/netology/blob/master/dk_cv3.ipynb)

# Классификация изображений: Cats vs Dogs
Обучить модель классификации изображение на 2 класса. Исходные данные и валидация решения на kaggle в рамках контеста Cats vs Dogs. Шаблон ipython-ноутбука для решения можно скачать по ссылке. Решения необходимо прислать в виде ipython-ноутбука с указанием значения метрики на Leaderboard. 

Задание засчитывается при значениях метрики Log Loss меньше 0.3.​

In [0]:
import warnings
#warnings.filterwarnings("ignore")
import numpy as np
import keras
from keras.models import Model
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import pandas as pd



In [0]:
!pip install kaggle-cli

In [0]:
!rm -rf ./train/ && rm -rf ./test/

In [0]:
!kg  download -u webotdel@mail.ru -p 64136413L -c dogs-vs-cats-redux-kernels-edition -f train.zip

In [0]:
!kg  download -u webotdel@mail.ru -p 64136413L -c dogs-vs-cats-redux-kernels-edition -f test.zip

In [0]:
!unzip -q train.zip && unzip -q test.zip

In [0]:
IMG_SIZE = (299, 299)  # размер входного изображения сети

In [0]:
import re
from random import shuffle
from glob import glob

train_files = glob('./train/*.jpg')
test_files = glob('./test/*.jpg')

# загружаем входное изображение и предобрабатываем
def load_image(path, target_size=IMG_SIZE):
    img = load_img(path, target_size=target_size)  # загрузка и масштабирование изображения
    array = img_to_array(img)
    return preprocess_input(array)  # предобработка для VGG16

# генератор для последовательного чтения обучающих данных с диска
def fit_generator(files, batch_size=32):
    while True:
        shuffle(files)
        for k in range(len(files) // batch_size):
            i = k * batch_size
            j = i + batch_size
            if j > len(files):
                j = - j % len(files)
            x = np.array([load_image(path) for path in files[i:j]])
            y = np.array([1. if re.match('.*/dog\.\d', path) else 0. for path in files[i:j]])
            yield (x, y)

# генератор последовательного чтения тестовых данных с диска
def predict_generator(files):
    while True:
        for path in files:
            yield np.array([load_image(path)])

# **Визуализируем примеры для обучения**

In [0]:
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

In [0]:
fig = plt.figure(figsize=(10, 5))
for i, path in enumerate(train_files[:10], 1):
    subplot = fig.add_subplot(i // 5 + 1, 5, i)
    plt.imshow(plt.imread(path));
    subplot.set_title('%s' % path.split('/')[-1]);

### Будем использовать Xception

In [0]:
base_model = Xception(include_top = False,
                   weights = 'imagenet',
                   input_shape = (IMG_SIZE[0], IMG_SIZE[1], 3))

In [0]:
base_model.summary()

### Добавляем полносвязный слой и замораживаем остальные.

In [0]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [0]:
for layer in base_model.layers:
    layer.trainable = False

In [0]:
model.compile(optimizer='adam', 
              loss='binary_crossentropy',  # функция потерь binary_crossentropy (log loss
              metrics=['accuracy'])

In [0]:
model.summary()

In [0]:
shuffle(train_files)  # перемешиваем обучающую выборку

train_val_split = 50  # число изображений в валидационной выборке

validation_data = next(fit_generator(train_files[:train_val_split], train_val_split))

# запускаем процесс обучения
model.fit_generator(fit_generator(train_files[train_val_split:]),  # данные читаем функцией-генератором
        steps_per_epoch=10,  # число вызовов генератора за эпоху
        epochs=5,  # число эпох обучения
        validation_data=validation_data)

In [0]:
pred = model.predict_generator(predict_generator(test_files), len(test_files), max_queue_size=500, verbose = 1)

In [0]:
%matplotlib inline
from matplotlib import pyplot as plt
fig = plt.figure(figsize=(20, 20))
for i, (path, score) in enumerate(zip(test_files[:4], pred[:4]), 1):
    subplot = fig.add_subplot(i // 5 + 1, 5, i)
    plt.imshow(plt.imread(path));
    subplot.set_title('%.3f' % score);

### Попробуем немного затюнить. Выводим все слои и замораживаем все, кроме последних двух блоков

In [0]:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)


In [0]:
for layer in model.layers[:115]:
   layer.trainable = False
for layer in model.layers[115:]:
   layer.trainable = True

#### Компилируем модель

In [0]:
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='binary_crossentropy')

In [0]:
model.fit_generator(fit_generator(train_files[train_val_split:]),  # данные читаем функцией-генератором
        steps_per_epoch=10,  # число вызовов генератора за эпоху
        epochs=100,  # число эпох обучения
        validation_data=validation_data)

In [0]:
pred = model.predict_generator(predict_generator(test_files), len(test_files), max_queue_size=500, verbose = 1)

In [0]:
with open('submit.txt', 'w') as dst:
    dst.write('id,label\n')
    for path, score in zip(test_files, pred):
        dst.write('%s,%f\n' % (re.search('(\d+)', path).group(0), score))

In [0]:
files.download('submit.txt')

## Score в Kaggle = 0.05564